## DBScan Model

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.ndimage import gaussian_filter
from skimage.filters import sobel
import segyio

In [10]:
# Preprocessing Function
def preprocess_data(data, sigma=1):
    """
    Preprocess seismic data for DBSCAN:
    - Apply Gaussian smoothing
    - Normalize to [0, 1]
    - Use Sobel filter to enhance edges
    """
    smoothed = gaussian_filter(data, sigma=sigma)
    normalized = (smoothed - np.min(smoothed)) / (np.max(smoothed) - np.min(smoothed))
    edges = sobel(normalized)
    return edges


In [11]:
# Apply DBSCAN
def detect_discontinuities_with_dbscan(data, eps=0.5, min_samples=10):
    """
    Use DBSCAN to detect discontinuities in preprocessed data.
    Parameters:
        data: 3D seismic data
        eps: DBSCAN epsilon parameter
        min_samples: DBSCAN minimum samples parameter
    Returns:
        labels: DBSCAN cluster labels reshaped to the original data dimensions.
    """
    data_flat = data.reshape(-1, 1)  # Flatten for DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels_flat = db.fit_predict(data_flat)
    return labels_flat.reshape(data.shape)


In [12]:
# Visualization
def plot_dbscan_results(original_data, dbscan_labels, slice_index=10):
    """
    Plot the original seismic slice and the DBSCAN clustering results.
    """
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Original Seismic Data")
    plt.imshow(original_data[slice_index], cmap="gray")

    plt.subplot(1, 2, 2)
    plt.title("DBSCAN Clustering")
    plt.imshow(dbscan_labels[slice_index], cmap="jet")
    plt.colorbar(label="Cluster Labels")
    plt.show()

In [13]:
# Visualization
def visualize_results(original, preprocessed, labels, time_slice):
    """
    Visualize original, preprocessed, and DBSCAN results.
    """
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Original Data")
    plt.imshow(original[time_slice], cmap="gray")

    plt.subplot(1, 3, 2)
    plt.title("Preprocessed (Edges)")
    plt.imshow(preprocessed[time_slice], cmap="gray")

    plt.subplot(1, 3, 3)
    plt.title("DBSCAN Clusters")
    plt.imshow(labels[time_slice], cmap="jet")
    plt.colorbar(label="Cluster Labels")
    plt.show()

In [14]:
filename_seismic = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Pp.sgy"
filename_ai = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Fault.sgy"
filename_fault = "C:\\Users\\hsk21\\Desktop\\MLcourse\\github\\Seismic-Fault-Detection-using-Convolutional-Neural-Network\\Equinor Synthetic model\\Equinor Synthetic model\\issap20_Fault.sgy"



In [15]:
# Note: the the xline header info location is at segyio.su.cdpx [181]
def segy2numpy(filename: str) -> np.array:
    with segyio.open(filename, xline=181) as segyfile:
        return segyio.tools.cube(segyfile)


In [16]:

seismic = segy2numpy(filename_seismic)

seismic_preprocessed = preprocess_data(seismic, sigma=2)

# Apply DBSCAN for discontinuity detection
eps = 0.3
min_samples = 5
dbscan_labels = detect_discontinuities_with_dbscan(seismic_preprocessed, eps=eps, min_samples=min_samples)

# Visualize results
visualize_results(seismic, seismic_preprocessed, dbscan_labels, time_slice=10)

MemoryError: 